# Assessment of drift in Cycle 4.
This notebook calculates global averages of T, S, and BGC state variables in the uppermost layer, upper 100 m, and throughout the depth.

In [1]:
import climtas.nci
import cosima_cookbook as cc
import numpy as np
import scipy
import matplotlib as plt
#climtas.nci.Client(malloc_trim_threshold='64kib')
from dask.distributed import Client
client = Client()
client

2022-12-13 11:35:37,270 - distributed.diskutils - INFO - Found stale lock file and directory '/local/v45/hh0162/tmp/dask-worker-space/worker-j8vxyn4r', purging
2022-12-13 11:35:37,271 - distributed.diskutils - INFO - Found stale lock file and directory '/local/v45/hh0162/tmp/dask-worker-space/worker-g7urag11', purging
2022-12-13 11:35:37,272 - distributed.diskutils - INFO - Found stale lock file and directory '/local/v45/hh0162/tmp/dask-worker-space/worker-xusai3zn', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 4
Total threads: 16,Total memory: 44.92 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40777,Workers: 4
Dashboard: /proxy/8787/status,Total threads: 16
Started: Just now,Total memory: 44.92 GiB
Comm: tcp://127.0.0.1:35913,Total threads: 4
Dashboard: /proxy/34793/status,Memory: 11.23 GiB
Nanny: tcp://127.0.0.1:42915,


In [2]:
session = cc.database.create_session()

### What is the name of your ACCESS-OM2 experiment?

In [3]:
ename = '01deg_jra55v140_iaf_cycle4'

### Load the grid cell area info (needed for spatial averaging because grid cells have different sizes)

In [4]:
area = cc.querying.getvar(ename, 'area_t', session, n=1)
area

<xarray.DataArray 'area_t' (yt_ocean: 2700, xt_ocean: 3600)>
dask.array<open_dataset-a3dcdada652a62056900287409cd22dcarea_t, shape=(2700, 3600), dtype=float32, chunksize=(540, 720), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean  (yt_ocean) float64 -81.11 -81.07 -81.02 ... 89.89 89.94 89.98
Attributes:
    long_name:     tracer cell area
    units:         m^2
    valid_range:   [0.e+00 1.e+15]
    cell_methods:  time: point
    coordinates:   geolon_t geolat_t
    ncfiles:       ['/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_j...
    contact:       Andrew Kiss
    email:         andrew.kiss@anu.edu.au
    created:       2022-04-27
    description:   0.1 degree ACCESS-OM2 global model configuration under int...
    notes:         Run configuration and history: https://github.com/COSIMA/0...

In [5]:
dzt = cc.querying.getvar(ename, 'dzt', session, ncfile='%monthly%')
dzt

<xarray.DataArray 'dzt' (time: 732, st_ocean: 75, yt_ocean: 2700, xt_ocean: 3600)>
dask.array<concatenate, shape=(732, 75, 2700, 3600), dtype=float32, chunksize=(1, 19, 135, 180), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean  (yt_ocean) float64 -81.11 -81.07 -81.02 ... 89.89 89.94 89.98
  * st_ocean  (st_ocean) float64 0.5413 1.681 2.94 ... 5.511e+03 5.709e+03
  * time      (time) datetime64[ns] 1958-01-16T12:00:00 ... 2018-12-16T12:00:00
Attributes: (12/13)
    long_name:      t-cell thickness
    units:          m
    valid_range:    [-1.e+01  1.e+05]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t
    ...             ...
    ncfiles:        ['/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_...
    contact:        Andrew Kiss
    email:          andrew.kiss@anu.edu.au
    created:        2022-04-27
    description:    0.1 degree ACCESS-OM2 global model configuration under in...
    notes:          Run configuration and history: https://github.com/COSIMA/...

### Volume of grid cells (area multiplied by thickness)
do not add .load() or else the kernel will die.

In [6]:
vol = (dzt*area)
vol

<xarray.DataArray (time: 732, st_ocean: 75, yt_ocean: 2700, xt_ocean: 3600)>
dask.array<mul, shape=(732, 75, 2700, 3600), dtype=float32, chunksize=(1, 19, 135, 180), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean  (yt_ocean) float64 -81.11 -81.07 -81.02 ... 89.89 89.94 89.98
  * st_ocean  (st_ocean) float64 0.5413 1.681 2.94 ... 5.511e+03 5.709e+03
  * time      (time) datetime64[ns] 1958-01-16T12:00:00 ... 2018-12-16T12:00:00

### What are the names of the variables you want to analyse? The list is below:
http://cosima.org.au/index.php/2020/07/29/data-available-0-1-1958-2018-access-om2-iaf-run/

In [7]:
vname = ['temp', 'salt', 'o2', 'adic', 'dic', 'alk', 'caco3', 'det', 'fe', 'phy', 'zoo', 'no3']

### Specify the depth limits for vertical averaging
In practice, 10000 = None

In [8]:
fukasa = [1,100,10000]

### Do the loop to produce global average
Save the 61-year time series, so that plotting can be done separately.

In [ ]:
vol1y = vol.groupby('time.year').mean().load()


In [ ]:
vol1y = vol.groupby('time.year').mean().load()
for i in range(np.size(vname)):
    data = cc.querying.getvar(ename, vname[i], session, ncfile='%monthly%')
    data1y = data.groupby('time.year').mean().load()
    for j in range(np.size(fukasa)):
        dataz = data1y.sel(st_ocean=slice(None,fukasa[j]))
        volz = vol1y.sel(st_ocean=slice(None,fukasa[j]))
        ts1y = (dataz*volz).sum(('st_ocean','yt_ocean','xt_ocean'))/volz.sum(('st_ocean','yt_ocean','xt_ocean'))
        ts1y.to_netcdf('/g/data/v45/hh0162/projects/notebook_access-om2/nc/ACCESS-WOMBAT_01deg_BGC_validation-'+vname[i]+'-upper-'+str(fukasa[j])+'.nc')


### Plot the 61-year surface, upper-100m, whole-depth time series